# Test Environment 

In [ ]:
import gym
# Assuming your package is installed or files are in the right place
# Replace with your actual import path if needed
from drone_2d_custom_gym_env.drone_2d_env import Drone2dEnv
import time

# --- Configuration Options ---
RENDER = True
MAX_EPISODE_STEPS = 1500
MOVING_PLATFORM = False
PLATFORM_SPEED = 2.5
INITIAL_RANDOM_RANGE = 9.0 # +/- 10m range

def run_env():
    # Initialize environment with desired options
    env = Drone2dEnv(
        render_sim=RENDER,
        max_steps=MAX_EPISODE_STEPS,
        moving_platform=MOVING_PLATFORM,
        platform_speed=PLATFORM_SPEED,
        initial_pos_random_range_m=INITIAL_RANDOM_RANGE
    )

    print("Action Space:", env.action_space)
    print("Observation Space:", env.observation_space)
    print("Observation Space Sample:", env.observation_space.sample())

    # Run a few episodes
    for episode in range(100):
        obs = env.reset()
        done = False
        total_reward = 0
        step_count = 0
        print(f"\n--- Episode {episode + 1} Starting ---")

        while not done:
            if RENDER:
                env.render()
                # Add a small delay to make rendering watchable
                # time.sleep(0.01)

            # --- Replace with your RL agent's action selection ---
            # action = agent.predict(obs) # Example placeholder
            action = env.action_space.sample() # Take random actions for now
            # ---

            # Step the environment
            obs, reward, done, info = env.step(action)

            total_reward += reward
            step_count += 1

            # Optional: Print step info
            # if step_count % 50 == 0:
            #     print(f"Step: {step_count}, Reward: {reward:.3f}, Done: {done}")
            #     # print(f"Obs: {[f'{x:.2f}' for x in obs]}")
            #     # print(f"Info: {info}")

            if done:
                print(f"--- Episode {episode + 1} Finished ---")
                print(f"Steps: {step_count}, Total Reward: {total_reward:.2f}")
                print(f"Final Info: {info}")
                if RENDER:
                    # Keep window open briefly after episode ends
                    env.render()
                    time.sleep(1.5)

    env.close()
    print("\nEnvironment Closed.")

if __name__ == "__main__":
    run_env()

Observation Space Size: (11,)
Drone starting at: (24.38, 33.64) m
Environment Reset. Wind Enabled: True, Wind Dir: 12.1 deg. Moving Platform: False
Action Space: Box(-1.0, 1.0, (2,), float32)
Observation Space: Box([-1. -1. -1. -1. -1. -1.  0. -1. -1. -1.  0.], 1.0, (11,), float32)
Observation Space Sample: [ 0.6603573  -0.02156259  0.11387867  0.52196664 -0.19000152  0.7052587
  0.8861563  -0.48190346  0.07125058  0.46223328  0.6641413 ]
Drone starting at: (33.40, 39.55) m
Environment Reset. Wind Enabled: True, Wind Dir: 154.8 deg. Moving Platform: False

--- Episode 1 Starting ---
Step 97: OUT OF BOUNDS!
--- Episode 1 Finished ---
Steps: 98, Total Reward: -39.44
Final Info: {'Battery': np.float32(97.10521), 'landed': False, 'crashed': False, 'out_of_bounds': True, 'Battery_empty': False, 'lost_control': False, 'steps': 98, 'raw_pos': (31.089523033818182, 50.01374884244158), 'raw_vel': (-0.08934811298723061, 10.941039093220525), 'raw_angle_rad': -0.6428984199586334, 'raw_angular_vel':

In [1]:
# --- START OF FILE test_drone_env.py ---

import gym
import numpy as np
import time

# IMPORTANT: Adjust this import path based on how your project is structured
# and how you installed your custom environment.
# Option 1: If installed as a package
# from drone_2d_custom_gym_env_package.drone_2d_custom_gym_env import Drone2dEnv
# Option 2: If drone_2d_env.py is in the same directory or accessible via PYTHONPATH
try:
    from drone_2d_custom_gym_env.drone_2d_env import Drone2dEnv
except ImportError:
    print("Error importing Drone2dEnv. Make sure the environment file is accessible.")
    print("Attempting relative import...")
    # Adjust relative path if needed (e.g., if test file is outside the package)
    from drone_2d_custom_gym_env_package.drone_2d_custom_gym_env.drone_2d_env import Drone2dEnv


# --- Test Configuration ---
NUM_EPISODES = 5       # How many episodes to run
MAX_STEPS_PER_EPISODE = 750 # Max steps before terminating an episode
RENDER_SIM = True      # Set to True to visualize the simulation
RENDER_DELAY_S = 0.02  # Small delay (in seconds) between frames if rendering

# --- Environment Options ---
# Choose the configuration you want to test
ENABLE_WIND_TEST = True
MOVING_PLATFORM_TEST = True
PLATFORM_SPEED_TEST = 2.5 # Meters per second
INITIAL_RANDOM_RANGE_TEST = 10.0 # Meters (+/- from center)
MAX_TILT_ANGLE_DEG_TEST = 90.0 # Degrees

def run_test():
    """Runs the test loop for the Drone2dEnv."""
    print("--- Starting Drone Environment Test ---")

    # Convert degrees to radians for the environment parameter
    max_tilt_rad = np.radians(MAX_TILT_ANGLE_DEG_TEST)

    # Initialize the environment with chosen options
    try:
        env = Drone2dEnv(
            render_sim=RENDER_SIM,
            max_steps=MAX_STEPS_PER_EPISODE,
            moving_platform=MOVING_PLATFORM_TEST,
            platform_speed=PLATFORM_SPEED_TEST,
            initial_pos_random_range_m=INITIAL_RANDOM_RANGE_TEST,
            enable_wind=ENABLE_WIND_TEST,
            max_allowed_tilt_angle_rad=max_tilt_rad,
            # Keep other render options (path, shade) as their defaults or specify them
            # render_path=True,
            # render_shade=True,
        )
    except Exception as e:
        print(f"\n!!! Error creating environment: {e} !!!")
        print("Please check the environment's __init__ method and parameters.")
        return

    print("\n--- Environment Initialized ---")
    print(f"Action Space: {env.action_space}")
    print(f"Observation Space: {env.observation_space}")
    # print(f"Sample Observation: {env.observation_space.sample()}") # Can be useful for checking bounds

    total_steps_all_episodes = 0

    for episode in range(NUM_EPISODES):
        print(f"\n--- Starting Episode {episode + 1}/{NUM_EPISODES} ---")
        try:
            # Reset the environment for a new episode
            obs = env.reset()
            print(f"Initial Observation sample: {obs[:4]}...") # Print start of obs
        except Exception as e:
            print(f"\n!!! Error resetting environment: {e} !!!")
            print("Skipping episode.")
            continue

        done = False
        total_reward = 0
        step_count = 0

        while not done:
            # Render the current state (if enabled)
            if RENDER_SIM:
                try:
                    env.render()
                    if RENDER_DELAY_S > 0:
                        time.sleep(RENDER_DELAY_S)
                except Exception as e:
                    print(f"\n!!! Error rendering environment: {e} !!!")
                    print("Disabling rendering for this episode.")
                    # Optionally disable rendering completely: RENDER_SIM = False
                    break # Stop this episode if rendering fails badly

            # --- Action Selection ---
            # For testing, use random actions. Replace with your agent's policy later.
            action = env.action_space.sample()
            # print(f"Step {step_count}: Action = {action}") # Uncomment for debugging action values

            # --- Step the Environment ---
            try:
                obs, reward, done, info = env.step(action)
                total_reward += reward
                step_count += 1
                total_steps_all_episodes += 1
            except Exception as e:
                print(f"\n!!! Error during env.step() at step {step_count}: {e} !!!")
                print("Terminating episode.")
                done = True # Force episode termination
                info = info if 'info' in locals() else {} # Use existing info if available
                info['error'] = f"Exception during step: {e}"

            # Optional: Print step info periodically
            # if step_count % 100 == 0:
            #     print(f"  Step: {step_count}, Reward: {reward:.3f}, Done: {done}")

            # Check if max steps reached (env should handle this with 'done', but as a backup)
            if step_count >= MAX_STEPS_PER_EPISODE and not done:
                print("Warning: Max steps reached in test loop, but env not 'done'.")
                done = True # Force termination in test script

        # --- Episode End ---
        print(f"--- Episode {episode + 1} Finished ---")
        print(f"Steps taken: {step_count}")
        print(f"Total Reward: {total_reward:.2f}")
        print(f"Final Info: {info}")

        # Render the final frame after done (if rendering)
        if RENDER_SIM:
             try: env.render(); time.sleep(0.5) # Pause briefly on final frame
             except: pass # Ignore render errors on final frame

    # --- Cleanup ---
    try:
        env.close()
        print("\nEnvironment Closed.")
    except Exception as e:
        print(f"\nError closing environment: {e}")

    print(f"\n--- Test Finished ---")
    print(f"Total steps across all episodes: {total_steps_all_episodes}")


if __name__ == "__main__":
    run_test()

--- Starting Drone Environment Test ---
Observation Space Size: (12,)
Drone starting at: (26.38, 31.48) m
Environment Reset. Wind Enabled: True, Wind Dir: 342.6 deg. Moving Platform: True

--- Environment Initialized ---
Action Space: Box(-1.0, 1.0, (2,), float32)
Observation Space: Box([-1. -1. -1. -1. -1. -1.  0. -1. -1. -1.  0. -1.], 1.0, (12,), float32)

--- Starting Episode 1/5 ---
Drone starting at: (20.07, 49.44) m
Environment Reset. Wind Enabled: True, Wind Dir: 208.0 deg. Moving Platform: True
Initial Observation sample: [-0.19727859  0.97762793  0.          0.        ]...
Step 31: OUT OF BOUNDS!
--- Episode 1 Finished ---
Steps taken: 32
Total Reward: -45.08
Final Info: {'Battery': np.float32(99.10136), 'landed': False, 'crashed': False, 'out_of_bounds': True, 'Battery_empty': False, 'lost_control': False, 'steps': 32, 'raw_pos': (19.5987277888651, 50.03115793165703), 'raw_vel': (-1.4347040886907416, 1.9520962459144362), 'raw_angle_rad': 0.005874853668688493, 'raw_angular_vel

# Training the model (Agent)

## Training Configuration

In [ ]:
MOVING_PLATFORM: True               #  Train with moving platform
PLATFORM_SPEED: 2.0                 #  Platform speed if moving
   
INITIAL_POST_RAMDOM_RANGE_M: 8.0    #  Random start range
MAX_ALLOWED_TILT_ANGLE_RAD: 2       #  Allow slightly more tilt (approx 90 deg)
ENABLE_WIND: True                   #  Train with wind enabled
REWARD_LANDING: 200                 #  reward for landing more high reward increase interest   
REWARD_UN_LANDING: 50  

In [ ]:

from stable_baselines3 import PPO

# --- Environment Configuration ---
# These should match the parameters you want to train the agent on
ENV_CONFIG = {
    "render_sim": False,             # IMPORTANT: Keep False for faster training
    "max_steps": 750,                # Max steps per episode during training
    "render_path": False,            # Disable rendering options for speed
    "render_shade": False,
    "shade_distance_m": 2.0,
             
    # No change if we are going resume training training
    
    "moving_platform": MOVING_PLATFORM,        #  Train with moving platform
    "platform_speed": PLATFORM_SPEED,           #  Platform speed if moving
    "initial_pos_random_range_m": INITIAL_POST_RAMDOM_RANGE_M,#  Random start range
    "max_allowed_tilt_angle_rad": MAX_ALLOWED_TILT_ANGLE_RAD, #  Allow slightly more tilt (approx 90 deg)
    "enable_wind": ENABLE_WIND,             #  Train with wind enabled
    'reward_landing': REWARD_LANDING,           #  reward for landing more high reward increase interest   
    'reward_un_landing': REWARD_UN_LANDING          #  reward for unstable landing more high reward increase interest   
    # Note: wind_speed is fixed at 5.0 inside the env for now
}


# --- Training Configuration ---
TOTAL_TIMESTEPS = 1_800_000      # Total steps for training (adjust as needed)
MODEL_ALGORITHM = PPO           # Algorithm to use (PPO is a good default)
POLICY_TYPE = "MlpPolicy"       # Policy type (Multi-Layer Perceptron for vector observations)
LOG_DIR = "logs/drone_ppo/"     # Directory to save TensorBoard logs
MODEL_SAVE_PATH = "models/ppo_drone_second" # Path to save the trained model
CHECKPOINT_FREQ = 50000         # Save a checkpoint every N steps
N_ENVS = 10                      # Number of parallel environments (adjust based on CPU cores)


## Training

In [27]:

import os
import gym

from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.vec_env import DummyVecEnv, SubprocVecEnv
from stable_baselines3.common.callbacks import CheckpointCallback

# --- Environment Import ---
# IMPORTANT: Adjust this import path based on your project structure
from drone_2d_custom_gym_env.drone_2d_env import Drone2dEnv



if __name__ == "__main__":
    # --- Setup ---
    os.makedirs(LOG_DIR, exist_ok=True)
    os.makedirs(os.path.dirname(MODEL_SAVE_PATH), exist_ok=True)

    print("--- Starting Drone Training ---")
    print(f"Algorithm: {MODEL_ALGORITHM.__name__}")
    print(f"Total Timesteps: {TOTAL_TIMESTEPS}")
    print(f"Environment Config: {ENV_CONFIG}")
    print(f"Using {N_ENVS} parallel environments.")

    # --- Create Vectorized Environment ---
    # Use SubprocVecEnv for true parallelism, DummyVecEnv for debugging
    # `env_kwargs` passes the configuration dictionary to each environment instance
    env = make_vec_env(
        Drone2dEnv,
        n_envs=N_ENVS,
        seed=0,
        vec_env_cls=SubprocVecEnv, # Use SubprocVecEnv for parallel processing
        #vec_env_cls=DummyVecEnv, # Use DummyVecEnv for easier debugging
        env_kwargs=ENV_CONFIG
        )

    # --- Setup Model ---
    # Define the model with policy type, environment, and logging parameters
    # You might need to tune hyperparameters like learning_rate, n_steps, batch_size etc.
    model = MODEL_ALGORITHM(
        POLICY_TYPE,
        env,
        verbose=1, # Print training progress
        tensorboard_log=LOG_DIR,
        # Example of adjusting some hyperparameters (optional):
        # learning_rate=3e-4,
        # n_steps=2048, # Steps per env before update (adjust based on max_steps and N_ENVS)
        # batch_size=64,
        # gamma=0.99, # Discount factor
        # gae_lambda=0.95,
        # ent_coef=0.0, # Entropy coefficient
    )

    # --- Setup Checkpoint Callback ---
    # Saves the model periodically during training
    checkpoint_callback = CheckpointCallback(
        save_freq=max(CHECKPOINT_FREQ // N_ENVS, 1), # Adjust frequency based on N_ENVS
        save_path=LOG_DIR,
        name_prefix="drone_ppo_ckpt"
    )

    # --- Train the Agent ---
    print("\n--- Starting Training ---")
    try:
        model.learn(
            total_timesteps=TOTAL_TIMESTEPS,
            log_interval=10, # Log metrics every 10 updates
            tb_log_name=f"{MODEL_ALGORITHM.__name__}_Drone", # Name for TensorBoard run
            callback=checkpoint_callback # Add the checkpoint callback
            )
    except Exception as e:
        print(f"\n!!! Error during training: {e} !!!")
        print("Attempting to save model before exiting...")
        model.save(f"{MODEL_SAVE_PATH}_error")
    finally:
        # --- Save the Final Model ---
        print("\n--- Training Finished (or interrupted) ---")
        print(f"Saving final model to: {MODEL_SAVE_PATH}")
        model.save(MODEL_SAVE_PATH)

        # --- Clean up ---
        env.close() # Close the vectorized environment
        print("Environment closed.")

    print("\n--- Training Script Complete ---")
    print(f"To monitor training, run: tensorboard --logdir {LOG_DIR}")

# --- END OF FILE train_drone_agent.py ---


--- Starting Drone Training ---
Algorithm: PPO
Total Timesteps: 1800000
Environment Config: {'render_sim': False, 'max_steps': 750, 'render_path': False, 'render_shade': False, 'shade_distance_m': 2.0, 'moving_platform': True, 'platform_speed': 2.0, 'initial_pos_random_range_m': 8.0, 'max_allowed_tilt_angle_rad': 2, 'enable_wind': True}
Using 10 parallel environments.
Observation Space Size: (12,)Observation Space Size: (12,)Observation Space Size: (12,)

Observation Space Size: (12,)Observation Space Size: (12,)


Observation Space Size: (12,)
Drone starting at: (25.22, 46.51) m
Observation Space Size: (12,)
Drone starting at: (18.91, 44.42) m
Environment Reset. Wind Enabled: True, Wind Dir: 70.7 deg. Moving Platform: True
Environment Reset. Wind Enabled: True, Wind Dir: 288.7 deg. Moving Platform: True
Drone starting at: (29.66, 46.58) mDrone starting at: (21.80, 38.76) m

Environment Reset. Wind Enabled: True, Wind Dir: 270.3 deg. Moving Platform: True
Environment Reset. Wind Enable

/home/sebas_dev_linux/miniconda3/envs/drone_env/lib/python3.10/site-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(
/home/sebas_dev_linux/miniconda3/envs/drone_env/lib/python3.10/site-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(
/home/sebas_dev_linux/miniconda3/envs/drone_env/lib/python3.10/site-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium

Step 48: OUT OF BOUNDS!
Drone starting at: (28.41, 47.84) m
Environment Reset. Wind Enabled: True, Wind Dir: 4.7 deg. Moving Platform: True
Step 57: LOST CONTROL!
Drone starting at: (31.21, 35.61) m
Environment Reset. Wind Enabled: True, Wind Dir: 93.2 deg. Moving Platform: True
Step 63: LOST CONTROL!
Drone starting at: (31.68, 43.03) m
Environment Reset. Wind Enabled: True, Wind Dir: 273.3 deg. Moving Platform: True
Step 86: OUT OF BOUNDS!
Drone starting at: (17.60, 44.04) m
Environment Reset. Wind Enabled: True, Wind Dir: 141.6 deg. Moving Platform: True
Step 88: LOST CONTROL!
Drone starting at: (20.98, 35.00) m
Environment Reset. Wind Enabled: True, Wind Dir: 7.3 deg. Moving Platform: True
Step 90: LOST CONTROL!
Drone starting at: (24.25, 36.39) m
Environment Reset. Wind Enabled: True, Wind Dir: 180.6 deg. Moving Platform: True
Step 44: OUT OF BOUNDS!
Drone starting at: (28.52, 32.95) m
Environment Reset. Wind Enabled: True, Wind Dir: 301.8 deg. Moving Platform: True
Step 102: OUT O



**Explanation and How to Use:**

1.  **Save:** Save this code as a Python file (e.g., `train_drone_agent.py`) in your project, likely at the root level or wherever you keep your training scripts.
2.  **Adjust Import:** **Crucially**, fix the import path for `Drone2dEnv` at the top of the script to match your project's directory structure. The provided `try...except` block attempts common scenarios.
3.  **Configure Training:**
    *   `TOTAL_TIMESTEPS`: Set how long you want to train (1.8 million is a reasonable starting point, but might need more or less).
    *   `LOG_DIR`: Choose where TensorBoard logs will be saved.
    *   `MODEL_SAVE_PATH`: Choose where the final trained model `.zip` file will be saved.
    *   `CHECKPOINT_FREQ`: How often to save intermediate models (useful if training crashes).
    *   `N_ENVS`: Number of parallel environments. Start with `4` or match the number of CPU cores you have available. Using more environments generally speeds up training but uses more RAM.
4.  **Configure Environment:** Modify the `ENV_CONFIG` dictionary to set the desired parameters for the `Drone2dEnv` during training (wind, moving platform, start range, etc.). **Remember to keep `render_sim=False` for efficient training.**
5.  **Install Dependencies:** Make sure you have the necessary libraries installed in your environment:
    ```bash
    pip install stable-baselines3[extra] gym pygame pymunk numpy
    pip install 'shimmy>=2.0
    # Or if using gymnasium:
    # pip install stable-baselines3[extra] gymnasium pygame pymunk numpy
    ```
    *(You might already have these from setting up the environment)*
6.  **Run Training:** Open your terminal, navigate to the directory where you saved the script, and run:
    ```bash
    python train_drone_agent.py
    ```
7.  **Monitor with TensorBoard:** While training is running (or after it finishes), open *another* terminal, navigate to the same project directory (or one level above `LOG_DIR`), and run:
    ```bash
    tensorboard --logdir logs/drone_ppo/
    ```
    Then open the URL provided by TensorBoard (usually `http://localhost:6006/`) in your web browser to see graphs of the reward, loss functions, episode length, etc. This is essential for understanding if the agent is learning.
8.  **Tuning:** If the agent doesn't learn well, you may need to:
    *   Adjust the environment parameters (e.g., make the task easier initially by disabling wind or the moving platform).
    *   Tune the PPO hyperparameters within the `model = MODEL_ALGORITHM(...)` call (learning rate, rollout buffer size `n_steps`, etc.).
    *   Modify the reward function in `drone_2d_env.py` if the current shaping isn't effective.
    *   Train for longer (increase `TOTAL_TIMESTEPS`).

# Continuing/ resume training

## Configurations  - Continue Training 

In [40]:
# --- Training Configuration ---
# *** Paths ***
LOAD_MODEL_PATH = "models/ppo_drone_second_continued.zip" # <-- IMPORTANT: Path to the model saved previously
NEW_SAVE_PATH = "models/ppo_drone_second_continued_2" # <-- Path to save the model after *this* training session
LOG_DIR = "logs/drone_ppo/" # <-- Directory for TensorBoard logs (can be same or new)

# *** Training Parameters ***
ADDITIONAL_TIMESTEPS = 3_000_000 # <-- How many *more* steps to train
MODEL_ALGORITHM = PPO         # <-- Must match the algorithm of the loaded model
POLICY_TYPE = "MlpPolicy"     # <-- Must match the policy type of the loaded model
CHECKPOINT_FREQ = 50000       # Save a checkpoint every N steps (during this session)
N_ENVS = 10                    # Number of parallel environments (match previous setup if possible)

**Test Load**

In [41]:
from stable_baselines3 import PPO

# Attempt to load the saved model
model_loaded = PPO.load(LOAD_MODEL_PATH)
print("Model loaded successfully!")

Model loaded successfully!


## Continuing training

**Key features:**

*   Specifies the path to the model you want to load (`LOAD_MODEL_PATH`).
*   Specifies a *new* path to save the model after continued training (`NEW_SAVE_PATH`).
*   Specifies the number of *additional* timesteps to train for (`ADDITIONAL_TIMESTEPS`).
*   Uses the same environment configuration (`ENV_CONFIG`) - **important for compatibility**.
*   Sets `reset_num_timesteps=False` in `model.learn()` to ensure the training step count and logs continue correctly.
*   Includes error handling if the specified model to load doesn't exist.

In [34]:


import os
import gym
from stable_baselines3 import PPO  # Or the algorithm you used (e.g., SAC, TD3)
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.vec_env import DummyVecEnv, SubprocVecEnv
from stable_baselines3.common.callbacks import CheckpointCallback

# --- Environment Import ---

from drone_2d_custom_gym_env.drone_2d_env import Drone2dEnv



# --- Setup ---
os.makedirs(os.path.dirname(NEW_SAVE_PATH), exist_ok=True)
os.makedirs(LOG_DIR, exist_ok=True) # Log dir might already exist

print("--- Starting Drone Training Continuation ---")
print(f"Attempting to load model from: {LOAD_MODEL_PATH}")
print(f"Training for additional {ADDITIONAL_TIMESTEPS} timesteps.")
print(f"Environment Config: {ENV_CONFIG}")

# --- Check if Model Exists ---
if not os.path.exists(LOAD_MODEL_PATH):
    print(f"\n!!! Error: Model file not found at {LOAD_MODEL_PATH} !!!")
    print("Cannot continue training without a model to load.")
    exit()

# --- Create Vectorized Environment ---
# Needs to be created *before* loading the model
try:
    env = make_vec_env(
        Drone2dEnv,
        n_envs=N_ENVS,
        seed=0, # Seed can be different, but structure should be same
        vec_env_cls=SubprocVecEnv,
        env_kwargs=ENV_CONFIG
    )
except Exception as e:
    print(f"\n!!! Error creating environment: {e} !!!")
    exit()

# --- Load Model ---
try:
    print(f"\nLoading model...")
    # Pass the environment and tensorboard log directory to load
    # SB3 automatically detects the algorithm and policy type from the zip file
    model = MODEL_ALGORITHM.load(
        LOAD_MODEL_PATH,
        env=env, # Link the loaded model to the (new) environment instances
        tensorboard_log=LOG_DIR # Tell SB3 where to continue logging
    )
    print(f"Model loaded successfully. Current Timesteps: {model.num_timesteps}")
except Exception as e:
    print(f"\n!!! Error loading model from {LOAD_MODEL_PATH}: {e} !!!")
    print("Check if the file exists, is a valid SB3 model, and matches the environment structure.")
    env.close()
    exit()

# --- Setup Checkpoint Callback ---
checkpoint_callback = CheckpointCallback(
    save_freq=max(CHECKPOINT_FREQ // N_ENVS, 1),
    save_path=LOG_DIR, # Save checkpoints in the log directory
    name_prefix=f"{os.path.basename(NEW_SAVE_PATH)}_ckpt" # Prefix based on new save name
)

# --- Continue Training ---
print("\n--- Continuing Training ---")
start_timesteps = model.num_timesteps # Get current steps from loaded model
target_timesteps = start_timesteps + ADDITIONAL_TIMESTEPS
print(f"Training from {start_timesteps} to {target_timesteps} total steps.")

try:
    model.learn(
        total_timesteps=ADDITIONAL_TIMESTEPS, # Train for the *additional* steps
        log_interval=10,
        tb_log_name=f"{MODEL_ALGORITHM.__name__}_Drone", # Use same or new log name
        reset_num_timesteps=False, # <-- IMPORTANT: Do NOT reset timestep counter
        callback=checkpoint_callback
        )
except Exception as e:
    print(f"\n!!! Error during continued training: {e} !!!")
    print("Attempting to save model before exiting...")
    model.save(f"{NEW_SAVE_PATH}_error")
finally:
    # --- Save the Final Model ---
    print("\n--- Training Finished (or interrupted) ---")
    print(f"Saving final model to: {NEW_SAVE_PATH}.zip") # SB3 adds .zip automatically
    model.save(NEW_SAVE_PATH) # Use the NEW save path

    # --- Clean up ---
    env.close()
    print("Environment closed.")

print("\n--- Training Continuation Script Complete ---")
print(f"To monitor training, run: tensorboard --logdir {LOG_DIR}")




--- Starting Drone Training Continuation ---
Attempting to load model from: models/ppo_drone_second.zip
Training for additional 3000000 timesteps.
Environment Config: {'render_sim': False, 'max_steps': 750, 'render_path': False, 'render_shade': False, 'shade_distance_m': 2.0, 'moving_platform': True, 'platform_speed': 2.0, 'initial_pos_random_range_m': 8.0, 'max_allowed_tilt_angle_rad': 2, 'enable_wind': True}
Observation Space Size: (12,)Observation Space Size: (12,)
Observation Space Size: (12,)

Observation Space Size: (12,)
Observation Space Size: (12,)
Observation Space Size: (12,)
Observation Space Size: (12,)
Observation Space Size: (12,)
Drone starting at: (17.80, 42.55) m
Drone starting at: (19.42, 42.40) m
Environment Reset. Wind Enabled: True, Wind Dir: 98.4 deg. Moving Platform: True
Environment Reset. Wind Enabled: True, Wind Dir: 236.0 deg. Moving Platform: True
Drone starting at: (32.95, 35.25) m
Drone starting at: (30.69, 39.52) m
Environment Reset. Wind Enabled: True, 

/home/sebas_dev_linux/miniconda3/envs/drone_env/lib/python3.10/site-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(
/home/sebas_dev_linux/miniconda3/envs/drone_env/lib/python3.10/site-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(
/home/sebas_dev_linux/miniconda3/envs/drone_env/lib/python3.10/site-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium

Step 203: CRASHED on ground!
Drone starting at: (28.65, 32.25) m
Environment Reset. Wind Enabled: True, Wind Dir: 276.9 deg. Moving Platform: True
Step 408: OUT OF BOUNDS!
Drone starting at: (26.46, 40.48) m
Environment Reset. Wind Enabled: True, Wind Dir: 123.7 deg. Moving Platform: True
Step 582: OUT OF BOUNDS!
Drone starting at: (19.84, 39.68) m
Environment Reset. Wind Enabled: True, Wind Dir: 88.6 deg. Moving Platform: True
Step 676: LOST CONTROL!
Drone starting at: (18.89, 39.38) m
Environment Reset. Wind Enabled: True, Wind Dir: 109.0 deg. Moving Platform: True
Step 679: OUT OF BOUNDS!
Drone starting at: (24.64, 38.49) m
Environment Reset. Wind Enabled: True, Wind Dir: 118.2 deg. Moving Platform: True
Step 704: OUT OF BOUNDS!
Drone starting at: (21.40, 39.39) m
Environment Reset. Wind Enabled: True, Wind Dir: 23.9 deg. Moving Platform: True
Step 750: MAX STEPS REACHED!
Step 750: MAX STEPS REACHED!
Step 750: MAX STEPS REACHED!
Step 750: MAX STEPS REACHED!
Drone starting at: (20.34


**Explanation and How to Use:**

1.  **Save:** Save this code as `continue_training.py` (or similar) in your project.
2.  **Configure Paths:**
    *   Set `LOAD_MODEL_PATH` to the exact path of the `.zip` file generated by your previous training run (e.g., `models/ppo_drone_final.zip`).
    *   Set `NEW_SAVE_PATH` to where you want the model saved *after this continuation*. It's generally good practice to give it a distinct name initially (e.g., `models/ppo_drone_final_continued`).
    *   Set `LOG_DIR` (usually the same as before, so TensorBoard shows one continuous graph).
3.  **Configure Training:**
    *   Set `ADDITIONAL_TIMESTEPS` to the number of *extra* steps you want to train for.
    *   Ensure `MODEL_ALGORITHM` matches the algorithm used to create the loaded model (the script will load it automatically, but it's good practice).
4.  **Configure Environment (`ENV_CONFIG`):** Make sure this dictionary matches the settings used during the *original* training run, or at least is compatible (same observation/action space dimensions).
5.  **Run:** Execute the script from your terminal:
    ```bash
    python continue_training.py
    ```
6.  **Monitor:** Use TensorBoard as before, pointing it to the `LOG_DIR`. You should see the training metrics continue from where the previous run left off.

This script provides a robust way to resume training your drone landing agent.

# Evaluation Agent

## Evaluation Configuration

In [ ]:
# --- Evaluation Configuration ---
# *** Paths ***
MODEL_PATH = "models/ppo_drone_second_continued.zip" # <-- IMPORTANT: Path to the trained model to evaluate
# MODEL_PATH = "logs/drone_ppo/drone_ppo_ckpt_100000_steps.zip" # Example for loading a checkpoint

# *** Evaluation Parameters ***
NUM_EVAL_EPISODES = 2000        # How many episodes to run for evaluation
RENDER_SIM = False             # Set to True to watch the agent perform
#RENDER_SIM = False             # Set to True to watch the agent perform
#RENDER_DELAY_S = 0.03         # Delay between frames when rendering (seconds)
RENDER_DELAY_S = -0.5         # Delay between frames when rendering (seconds)

# *** Environment Configuration for Evaluation ***
# Use settings representative of how you expect the agent to perform,
# or the specific conditions you want to test.
# Often similar or identical to training config, but render_sim=True here.
ENV_CONFIG_EVAL = {

    "render_sim": RENDER_SIM,         # Use the RENDER_SIM flag here
    "max_steps": 1000,                # Allow more steps for evaluation if needed
    "render_path": True,              # Enable path/shade rendering for visualization
    "render_shade": True,
    "shade_distance_m": 2.0,
             
    # No if we are going to evaluate same conditions
    
    "moving_platform": MOVING_PLATFORM,        #  Train with moving platform
    "platform_speed": PLATFORM_SPEED,           #  Platform speed if moving
    "initial_pos_random_range_m": INITIAL_POST_RAMDOM_RANGE_M,#  Random start range
    "max_allowed_tilt_angle_rad": MAX_ALLOWED_TILT_ANGLE_RAD, #  Allow slightly more tilt (approx 90 deg)
    "enable_wind": ENABLE_WIND,             #  Train with wind enabled
    'reward_landing': REWARD_LANDING,           #  reward for landing more high reward increase interest   
    'reward_un_landing': REWARD_UN_LANDING          #  reward for unstable landing more high reward increase interest    
    # Note: wind_speed is fixed at 5.0 inside the env for now
}

In [ ]:


import os
import gym
import numpy as np
import time
import pygame # Import pygame directly for flip
from stable_baselines3 import PPO # Or the algorithm you used (e.g., SAC, TD3)
# REMOVED: from stable_baselines3.common.vec_env import DummyVecEnv

# --- Environment Import ---
from drone_2d_custom_gym_env.drone_2d_env import Drone2dEnv




# --- Helper Function --- MODIFIED
def evaluate_agent(model, env, num_episodes=10, render=False, delay=0.0):
    """
    Evaluates a Stable Baselines3 agent on a single environment instance.

    :param model: The agent to evaluate.
    :param env: The single environment instance. # Changed comment
    :param num_episodes: Number of episodes to run.
    :param render: Whether to render the environment.
    :param delay: Delay between frames if rendering.
    :return: Dictionary containing evaluation statistics.
    """
    episode_rewards = []
    episode_lengths = []
    successes = 0
    failures = {"crashed": 0, "lost_control": 0, "out_of_bounds": 0, "battery_empty": 0, "timeout": 0, "other_error": 0}

    # Get clock and fps if rendering and available
    the_clock = None
    render_fps = 50
    if render and hasattr(env, 'clock') and env.clock:
        the_clock = env.clock
    if render and hasattr(env, 'metadata') and 'render_fps' in env.metadata:
         render_fps = env.metadata['render_fps']

    for episode in range(num_episodes):
        # Reset the single environment
        obs = env.reset()
        done = False # Single boolean flag now
        current_episode_reward = 0
        current_episode_length = 0
        final_info_dict = {}

        print(f"  Starting Eval Episode {episode + 1}/{num_episodes}...")

        while not done:
            if render:
                try:
                    env.render() # Call env's render to DRAW

                    # --- ADDED Display Update and Clock Tick HERE ---
                    pygame.display.flip() # Update the display
                    if the_clock:
                        the_clock.tick(render_fps) # Control FPS
                    # --- End Added Code ---

                    if delay > 0: time.sleep(delay)

                except Exception as e:
                    print(f"Error rendering/updating display: {e}"); render = False

            # Use deterministic actions
            action, _states = model.predict(obs, deterministic=True)

            try:
                # Step the single environment
                obs, reward, done, info = env.step(action) # No more lists/arrays

                current_episode_reward += reward
                current_episode_length += 1
                final_info_dict = info # Store the direct info dict

            except Exception as e:
                 print(f"Error during env.step(): {e}. Terminating episode.")
                 done = True
                 final_info_dict['error'] = f"Exception during step: {e}"
                 if 'other_error' not in failures: failures['other_error'] = 0
                 failures["other_error"] += 1

        # --- Episode Finished ---
        episode_rewards.append(current_episode_reward)
        episode_lengths.append(current_episode_length)

        # Check outcome (same logic using final_info_dict)
        if final_info_dict.get('landed_safely', False): successes += 1; print(f"  Episode {episode + 1}: SUCCESS (Landed Safely)")
        elif final_info_dict.get('crashed', False): failures["crashed"] += 1; print(f"  Episode {episode + 1}: FAILED (Crashed)")
        elif final_info_dict.get('lost_control', False): failures["lost_control"] += 1; print(f"  Episode {episode + 1}: FAILED (Lost Control)")
        elif final_info_dict.get('out_of_bounds', False): failures["out_of_bounds"] += 1; print(f"  Episode {episode + 1}: FAILED (Out of Bounds)")
        elif final_info_dict.get('Battery_empty', False): failures["battery_empty"] += 1; print(f"  Episode {episode + 1}: FAILED (Battery Empty)")
        elif current_episode_length >= env.max_steps: # Direct access to max_steps
            if not (final_info_dict.get('crashed', False) or final_info_dict.get('lost_control', False) or final_info_dict.get('out_of_bounds', False) or final_info_dict.get('Battery_empty', False) or final_info_dict.get('landed_safely', False)):
                failures["timeout"] += 1; print(f"  Episode {episode + 1}: FAILED (Timeout)")
        elif 'error' not in final_info_dict and not final_info_dict.get('landed_safely', False) :
             if 'other_error' not in failures: failures['other_error'] = 0; failures["other_error"] += 1; print(f"  Episode {episode + 1}: FAILED (Unknown)")

        print(f"  Episode {episode + 1}: Length={current_episode_length}, Reward={current_episode_reward:.2f}")

    # --- Calculate Statistics --- (same logic)
    mean_reward = np.mean(episode_rewards) if episode_rewards else 0; std_reward = np.std(episode_rewards) if episode_rewards else 0
    mean_length = np.mean(episode_lengths) if episode_lengths else 0; success_rate = successes / num_episodes if num_episodes > 0 else 0
    stats = {"mean_reward": mean_reward, "std_reward": std_reward, "mean_length": mean_length, "success_rate": success_rate, "successes": successes, "failures": failures, "total_episodes": num_episodes}
    return stats


if __name__ == "__main__":
    print("--- Starting Drone Agent Evaluation ---")
    print(f"Loading model from: {MODEL_PATH}")
    print(f"Evaluating for {NUM_EVAL_EPISODES} episodes.")
    print(f"Evaluation Environment Config: {ENV_CONFIG_EVAL}")

    if not os.path.exists(MODEL_PATH):
        print(f"\n!!! Error: Model file not found at {MODEL_PATH} !!!")
        exit()

    # --- Create SINGLE Evaluation Environment --- MODIFIED ---
    try:
        # Instantiate the environment directly
        eval_env = Drone2dEnv(**ENV_CONFIG_EVAL)
        print("Single evaluation environment created.")
    except Exception as e:
        print(f"\n!!! Error creating evaluation environment: {e} !!!")
        exit()
    # --- End Modification ---

    # --- Load Model ---
    try:
        # Pass the single env to load (SB3 handles wrapping internally if needed by model, but not needed here)
        loaded_model = PPO.load(MODEL_PATH, env=eval_env)
        print("\nModel loaded successfully.")
    except Exception as e:
        print(f"\n!!! Error loading model: {e} !!!")
        eval_env.close()
        exit()

    # --- Run Evaluation ---
    print("\n--- Running Evaluation Loop ---")
    eval_stats = evaluate_agent( # Pass the single env instance
        model=loaded_model,
        env=eval_env,
        num_episodes=NUM_EVAL_EPISODES,
        render=RENDER_SIM, # Pass the flag
        delay=RENDER_DELAY_S
    )

    # --- Print Summary --- (same logic)
    print("\n\n--- Evaluation Summary ---")
    print(f"Evaluated for {eval_stats['total_episodes']} episodes.")
    print(f"Mean Reward: {eval_stats['mean_reward']:.2f} +/- {eval_stats['std_reward']:.2f}")
    print(f"Mean Episode Length: {eval_stats['mean_length']:.1f}")
    print(f"Success Rate (Landed Safely): {eval_stats['success_rate']:.2%}")
    print(f"Successful Landings: {eval_stats['successes']}")
    print("Failure Breakdown:")
    for reason, count in eval_stats['failures'].items():
        if count > 0: print(f"  - {reason.replace('_', ' ').title()}: {count}")
    print("--------------------------")

    # --- Cleanup ---
    eval_env.close()
    print("Evaluation environment closed.")
    print("--- Evaluation Script Complete ---")



--- Starting Drone Agent Evaluation ---
Loading model from: models/ppo_drone_second_continued.zip
Evaluating for 2000 episodes.
Evaluation Environment Config: {'render_sim': False, 'max_steps': 1000, 'render_path': True, 'render_shade': True, 'shade_distance_m': 2.0, 'moving_platform': True, 'platform_speed': 2.0, 'initial_pos_random_range_m': 8.0, 'max_allowed_tilt_angle_rad': 1.5, 'enable_wind': True, 'reward_landing': 500, 'reward_un_landing': 50}

!!! Error creating evaluation environment: Drone2dEnv.__init__() got an unexpected keyword argument 'reward_landing' !!!
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.

Model loaded successfully.

--- Running Evaluation Loop ---
Drone starting at: (20.76, 38.27) m
Environment Reset. Wind Enabled: True, Wind Dir: 206.5 deg. Moving Platform: True
  Starting Eval Episode 1/2000...


/home/sebas_dev_linux/miniconda3/envs/drone_env/lib/python3.10/site-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(
/home/sebas_dev_linux/miniconda3/envs/drone_env/lib/python3.10/site-packages/stable_baselines3/common/on_policy_algorithm.py:150: UserWarning: You are trying to run PPO on the GPU, but it is primarily intended to run on the CPU when not using a CNN policy (you are using ActorCriticPolicy which should be a MlpPolicy). See https://github.com/DLR-RM/stable-baselines3/issues/1245 for more info. You can pass `device='cpu'` or `export CUDA_VISIBLE_DEVICES=` to force using the CPU.Note: The model will train, but the GPU utilization will be poor and the training might take longer than on CPU.
  warnings.warn(


Step 223: CRASHED on ground!
Step 223: LANDED SAFELY!
  Episode 1: FAILED (Crashed)
  Episode 1: Length=224, Reward=222.97
Drone starting at: (22.43, 32.20) m
Environment Reset. Wind Enabled: True, Wind Dir: 203.6 deg. Moving Platform: True
  Starting Eval Episode 2/2000...
Step 634: CRASHED on ground!
Step 634: CRASHED on pad! Speed: 2.17, Angle: 0.02
  Episode 2: FAILED (Crashed)
  Episode 2: Length=635, Reward=703.48
Drone starting at: (31.21, 46.86) m
Environment Reset. Wind Enabled: True, Wind Dir: 228.5 deg. Moving Platform: True
  Starting Eval Episode 3/2000...
Step 182: CRASHED on ground!
Step 182: CRASHED on ground!
  Episode 3: FAILED (Crashed)
  Episode 3: Length=183, Reward=31.23
Drone starting at: (23.11, 40.15) m
Environment Reset. Wind Enabled: True, Wind Dir: 185.7 deg. Moving Platform: True
  Starting Eval Episode 4/2000...
Step 1000: MAX STEPS REACHED!
  Episode 4: FAILED (Timeout)
  Episode 4: Length=1001, Reward=1356.07
Drone starting at: (31.48, 37.66) m
Environme

: 